In [ ]:

##1. identification of GOGs


#input file: Ortholouges.tsv from orthofinder output directory

##analysis of all eukaryote clustering - to get green protein families

import glob
import pandas as pd
import glob
import numpy as np
import os
import json
os.chdir(r'/path/to/working/directory')

with open ('688SP_code_to_name.txt') as file:
    data = file.read()

spcode_to_name_dic=json.loads(data) #dictionary between species code and species name (e.g. hsa for Homo Sapien) 

df = pd.read_csv('Orthogroups.GeneCount.tsv',sep = '\t') ##from orthofinder output
spcodelist=df.columns.tolist()
spcodelist=spcodelist[1:-1]
spnamelist=[]
for c in spcodelist:
    spnamelist.append(spcode_to_name_dic.get(c))

df0 = pd.read_csv('Species_taxonomy.csv')  #file with taxonomy including a col with green non green (Table S1)
dic = dict(zip(df0.Organism_Full_Name, df0.Green_non_Green))

Spcategorylist =['OG']
for sp in spnamelist:
    Spcategorylist.append(dic.get(sp))

Spcategorylist.append('Total')
df.columns=Spcategorylist  

df2 = df.groupby(df.columns, axis=1).sum()
df2['%_in green'] = df2['Green']*100/df2['Total']
df2 = df2.sort_values(by='%_in green', ascending=False)
df2['%_in non-green'] = df2['Non_green']*100/df2['Total']
df2.to_csv('OGs_protein_counts_green_vs_nongreen.csv', index=None)     #csv with details of  what % of proteins in a given OG is  green vs non green - this however doesnt normalise for multiple prots per species, for that we need to impliment binary matrix

df =df.set_index('OG')
df[df != 0] = 1
df['Total'] = df.sum(axis=1)
total_green_species = Spcategorylist.count('Green')
total_Non_green_species = Spcategorylist.count('Non_green')
df2 = df.groupby(df.columns, axis=1).sum()
df2=df2.drop(['Total'], axis=1)
df2['%_presence_in_green'] = df2['Green']*100/total_green_species
df2['%_presence_in_non_green'] = df2['Non_green']*100/total_Non_green_species

##criteria for green protein family - it should have representative present in at least 3 green and less than 3 non green
df3=df2[(df2['Green'] > 2) & (df2['Non_green'] < 3)]
df3['OGID']=df3.index

with open ('688SP_code_to_name.txt') as file:
    data = file.read()

spcode_to_name_dic=json.loads(data)

df4 = pd.read_csv('Orthogroups.GeneCount.tsv',sep = '\t')
df4.rename(columns=spcode_to_name_dic, inplace=True)
df5=df3.merge(df4, how='left', left_on='OGID', right_on='Orthogroup')
df5.to_csv('GOGs_GCN.csv', index=None)  ##table S1C

df4 = pd.read_csv('Orthogroups.tsv',sep = '\t')
df4.rename(columns=spcode_to_name_dic, inplace=True)
df5=df3.merge(df4, how='left', left_on='OGID', right_on='Orthogroup')
df5.to_csv('GOGs_protIDs.csv', index=None)  ##table S1D
